In [2]:
import requests
import base64
import pandas as pd
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup as bs4

In [3]:
load_dotenv()
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

# Encode client_id:client_secret in Base64
auth_str = f"{CLIENT_ID}:{CLIENT_SECRET}"
b64_auth_str = base64.b64encode(auth_str.encode()).decode()

url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": f"Basic {b64_auth_str}",
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {"grant_type": "client_credentials"}

response = requests.post(url, headers=headers, data=data)
token = response.json().get("access_token")
print(token)

BQDtqjfYB0JUqwsh1Y6W8RwTXWQuKQFfetV6wy12q8QRz5f9Ikq4f5DOI7Q_eC7wSXnNYnv0JKeHsTWLqVvswar2vKhJgQNa5ZbulCOYBADd8UVB8M0PZ-BFdLMo8Y-TH9YrYOI5AJY


In [4]:
ACCESS_TOKEN = token
playlist_id = '6UeSakyzhiEt4NB3UAd6NQ?si=GzuWZxKKSsagMcd6rjPOpw'

url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

response = requests.get(url, headers=headers)
data = response.json()

In [5]:
data

{'collaborative': False,
 'description': 'The official <a href="http://www.billboard.com/charts/hot-100">Billboard Hot 100</a> features this week\'s most popular songs across all genres, ranked by radio airplay monitored by Nielsen BDS, download sales tracked by Nielsen SoundScan and streaming activity data provided by leading online music services. For more information go to <a href="http://Billboard.com">Billboard.com</a>.',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6UeSakyzhiEt4NB3UAd6NQ'},
 'followers': {'href': None, 'total': 1802858},
 'href': 'https://api.spotify.com/v1/playlists/6UeSakyzhiEt4NB3UAd6NQ',
 'id': '6UeSakyzhiEt4NB3UAd6NQ',
 'images': [{'height': None,
   'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000da843bd5501a335b265807df34db',
   'width': None}],
 'name': 'Billboard Hot 100',
 'owner': {'display_name': 'Billboard',
  'external_urls': {'spotify': 'https://open.spotify.com/user/billboard.com'},
  'href': 'https://api.spotify.

In [25]:
track_list = []

# Menentukan jumlah artis maksimum dalam data
max_artists = max(len(item["track"]["artists"]) for item in data["tracks"]["items"])

for item in data["tracks"]["items"]:
    track = item["track"]
    album_name = track["album"]["name"]
    release_date = track["album"]["release_date"]
    track_name = track["name"]
    track_id = track['id']
    
    # List artis dan ID artis
    artist_names = [artist["name"] for artist in track["artists"]]
    artist_ids = [artist["id"] for artist in track["artists"]]

    # Mengisi artis yang kurang dari max_artists dengan None
    while len(artist_names) < max_artists:
        artist_names.append(None)
        artist_ids.append(None)
    
    # Membuat dictionary untuk setiap track
    track_info = {
        "album": album_name,
        "release_date": release_date,
        "track": track_name,
    }
    
    # Menambahkan artis dan ID-nya ke dictionary
    for i in range(max_artists):
        track_info[f"artist_{i+1}"] = artist_names[i]
        track_info[f"id_artist_{i+1}"] = artist_ids[i]
    
    track_list.append(track_info)

In [26]:
df = pd.DataFrame(track_list)
df

,album,release_date,track,artist_1,id_artist_1,artist_2,id_artist_2,artist_3,id_artist_3,artist_4,id_artist_4
0,GNX,2024-11-22,luther (with sza),Kendrick Lamar,2YZyLoL8N0Wb9xBt1NhZWg,SZA,7tYKF4w9nC0nq9CsPZTHyP,None,None,None,None
1,Die With A Smile,2024-08-16,Die With A Smile,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,Bruno Mars,0du5cEVh5yTK9QJze8zA0C,None,None,None,None
2,Not Like Us,2024-05-04,Not Like Us,Kendrick Lamar,2YZyLoL8N0Wb9xBt1NhZWg,None,None,None,None,None,None
3,A Bar Song (Tipsy),2024-04-12,A Bar Song (Tipsy),Shaboozey,3y2cIKLjiOlp1Np37WiUdH,None,None,None,None,None,None
4,GNX,2024-11-22,tv off (feat. lefty gunplay),Kendrick Lamar,2YZyLoL8N0Wb9xBt1NhZWg,Lefty Gunplay,1jiZvw42D4oquLl24x2VWV,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
95,DeBÍ TiRAR MáS FOToS,2025-01-05,VOY A LLeVARTE PA PR,Bad Bunny,4q3ewBCX7sLwd24euuV69X,None,None,None,None,None,None
96,Texas,2024-11-15,Texas,Blake Shelton,1UTPBmNbXNTittyMJrNkvw,None,None,None,None,None,None
97,Tu Boda,2024-09-26,Tu Boda,Oscar Maydon,3l9G1G9MxH6DaRhwLklaf5,Fuerza Regida,0ys2OFYzWYB5hRDLCsBqxt,None,None,None,None
98,ExtraL (feat. Doechii),2025-02-21,ExtraL (feat. Doechii),JENNIE,250b0Wlc5Vk0CoUsaCY84M,Doechii,4E2rKHVDssGJm2SCDOMMJB,None,None,None,None


In [32]:
def get_artist_genre(artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        genres = ", ".join(data.get("genres", []))  # Menggabungkan genre menjadi string
        return genres if genres else None  # Jika kosong, ubah menjadi None
    return None  # Jika tidak ada genre


df["artist_1_genre"] = df["id_artist_1"].apply(get_artist_genre)

In [38]:
# Fungsi untuk scrapping genre dari Last.fm
def scrape_lastfm_genre(artist_name):
    formatted_name = artist_name.replace(" ", "+")  # Format nama artis untuk URL
    url = f"https://www.last.fm/music/{formatted_name}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/",
        "Connection": "keep-alive"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = bs4(response.text, "html.parser")
        
        # Cari elemen pertama dalam <ul class="tags-list tags-list--global">
        tag_element = soup.select_one("ul.tags-list.tags-list--global li.tag a")
        if tag_element:
            return tag_element.text.strip()  # Ambil teks genre pertama

    return None  # Jika tidak ditemukan

for index, row in df.iterrows():
    if row["artist_1_genre"] is None:  # Jika genre None, cari di Last.fm
        artist_name = row["artist_1"]
        genre = scrape_lastfm_genre(artist_name)
        df.at[index, "artist_1_genre"] = genre  # Update dataframe

In [51]:
df[df['track'] == "APT."]

,album,release_date,track,artist_1,id_artist_1,artist_2,id_artist_2,artist_3,id_artist_3,artist_4,id_artist_4,artist_1_genre
5,APT.,2024-10-18,APT.,ROSÉ,3eVa5w3URK5duf6eyVDbu9,Bruno Mars,0du5cEVh5yTK9QJze8zA0C,None,None,None,None,k-pop


In [45]:
df.to_csv("Billboard 100 Spotify.csv", index=False)